# MongoDB 02 - 파이썬 MongoDB
pyMongo는 MongoDB와 Python을 연동하기 위한 MongoDB API로 비교적 간편하다. 


# 데이터베이스, 컬렉션, 도큐먼트

MongoDB는 기본적으로 하나의 JSON 객체를 단위로 사용한다. 읽고 쓰는 단위가 되는 JSON 객체를 도큐먼트라고 한다.
여러 도큐먼트를 모아 놓은 것이 컬렉션이며, 컬렉션의 집합이 데이터베이스 이다.
즉, 다음과 같은 포함 관계가 있다고 할 수 있다.

$$데이터베이스 \supset 컬렉션  \supset  도큐먼트 $$

기존의 오라클이나 MySQL 같은 관계형 데이터베이스와 MongoDB를 비교해(대응 시켜) 보자면, 데이터베이스는 동일하고, 테이블은 컬렉션에, 레코드는 도큐먼트에 대응 된다.

관계형 DB | MongoDB
--|--
데이터베이스  | 데이터베이스 
테이블 | 컬렉션 
레코드 | 도큐먼트

데이터베이스는 데이터집합 전체를 의미한다는 점에서 유사하고, 관계형 DB의 테이블(레코드의 집합)과 컬렉션(도큐먼트의 집합)은 데이터의 집합이라는 점에서 대응된다.  

MongoDB의 도큐먼트는 데이터를 추가/삭제하는 단위라는 점에서 관계형 DB의 레코드(record)에 대응되지만, 조금만 더 살펴보면 사실 완전히 다르다. 관계형 DB에서 하나의 테이블안에 있는 레코드들은 모드 같은 형태를 가지고 있지만, 도큐먼트는 JSON으로 표현되는 서로 다른 다양한 형태의 데이터를 저장할 수 있다. 이 점이 바로 기존의 관계형 DB와 가장 큰 차이점이다. 


## 기본 사용법
pyMongo를 통해 MongoDB를 사용하는 과정은 대략 다음과 같은 단계를 거친다.

1. 연결: MongoDB 서버에 접속하여 커넥션을 얻는다 (서버의 주소와 포트 등을 지정)
1. 데이터베이스와 컬렉션에 연결: 커넥션을 통해 데이터베이와 컬랙션에 대한 핸들러를 얻는다.
1. 도큐멘트 읽기/쓰기: 도큐멘트를 검색, 삭제, 생성 등의 조작을 수행한다.
1. 커넥션 닫기: 필요한 작업이 완료되면 커넥션을 닫는다.

우선, 위 과정 전체가 어떻게 진행되는지 간단한 예제를 통해 전체적인 흐름을 살펴보자.

In [1]:
from pymongo import MongoClient

# 1. 커넥션(connection) 생성
client = MongoClient()
# 호스트와 포트를 지정
# client = MongoClient('localhost', 27017)
# client = MongoClient('mongodb://localhost:27017/')


# 2. 데이터베이스에 연결하고, 컬렉션을 얻는다.
db = client.test
# 혹은, 딕셔너리 형식으로 데이터베이스를 지정할 수 도 있다.
# db = client ['test]
coll = db.simple # 콜렉션


# 3. 도큐멘트를 추가하고 검색
coll.insert_one({'key':'value'})

# 4. 도큐멘트를 추가하고 검색
for d in coll.find(): 
    print (d)

# 4. 커넥션을 닫는다
client.close()

{'key': 'value', '_id': ObjectId('571c062d9974a856d8f2cb42')}
{'key': 'value', '_id': ObjectId('571c13bb9974a856d8f2cb53')}
{'key': 'value', '_id': ObjectId('571c13fa9974a8619974247e')}


# MongoDB에 연결
호스트 이름과 포트를 지정하여 MongoDB에 연결한다.

In [2]:
import pymongo

client = MongoClient('mongodb://localhost:27017/')

연결 실패에 대응하려면 다음과 같이 예외처리를 해주는 것이 좋다.

In [3]:
try:
    client = MongoClient('mongodb://localhost:27017/')
except pymongo.errors.ConnectionFailure as e:
    print ("Connection Fail: %s" % e)

# 데이터베이스
일단 연결을 얻으면 속성을 지정하는 형식으로 데이터베이를 지정할 수 있다. 만일 데이터베이스가 존재하지 않으면 자동으로 생성된다.

In [4]:
# 'stocks' 데이터베이스 얻기
db = client.stocks

# 딕셔너리 형식 conn['stocks'] 으로 쓸 수 도 있다.
db = client['stocks']

database_names() 메서드로 MongoDB 내에 데이터베이스 목록을 얻을 수 있다.

In [5]:
client.database_names()

['stocks', 'local', 'test', 'admin']

## 컬렉션
데이터베이스는 다수의 컬렉션을 포함할 수 있다. 
커넥션으로 부터 데이터베이스를 얻는 방법과 마찬가지로, 데이터베이스로 부터 컬렉션을 얻는다. 컬렉션이 존재하지 않으면 새로 생성된다. (데이터베이스나 컬렉션은 데이터를 최초 추가할 때 실제 생성된다)

In [6]:
coll = db.mystocks
coll

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'stocks'), 'mystocks')

## 도큐먼트
도큐먼트는 JOSN 문법으로 구성된 구조적 데이터를 말한다. 예를 들어 다음과 같은 형태의 JSON 데이터 구조가 하나의 도큐먼트가 될 수 있다.

```javascript
{ "Ticker": "AAPL", "Company":"Apple Inc."}
```

도큐먼트를 컬렉션에 저장하려면 insert_one() 혹은 replace_one() 메서드를 쓴다.

In [7]:
coll.insert_one({"Ticker": "AAPL"})
coll.insert_one({"Ticker": "GOOG"})

# find(), find_one()

저장된 내용을 확인해 보자. 저장된 내용을 검색하려면 find() 혹은 find_one()를 쓴다. 검색 결과로 다수의 도큐먼트를 얻으려면 find(), 단 한개의 도큐먼트를 얻으려면 find_one()을 사용한다. 

find()의 인자로 아무도 지정하지 않으면 컬렉션의 도큐먼트 전체를 반환한다.
컬렉션 내의 도큐먼트 전체를 출력해보려면 다음과 같이 한다.

In [8]:
for x in coll.find():
    print(x)

{'_id': ObjectId('571c13fa9974a8619974247f'), 'Ticker': 'AAPL'}
{'_id': ObjectId('571c13fa9974a86199742480'), 'Ticker': 'GOOG'}


find_one()은 처음 도큐먼트 1개를 반환한다. (검색 조건을 지정하지 않았으므로 컬렉션에 저장된 첫 도큐먼트가 된다)

In [9]:
x = coll.find_one() 
x

{'Ticker': 'AAPL', '_id': ObjectId('571c13fa9974a8619974247f')}

In [10]:
x = coll.find_one({"Ticker":"AAPL"})
x

{'Ticker': 'AAPL', '_id': ObjectId('571c13fa9974a8619974247f')}

# _id 필드
우리가 저장한 필드는 한 개(Ticker)였는데, "_id"라는 필드가 더 있는 것을 확인할 수 있다. 이 _id 필드는 새로 저장되는 시점에 생성되며 도큐먼트를 구분하는 도큐먼트 고유의 식별자이다 (컬렉션 내의 _id는 모두 고유한 값을 가진다).

In [11]:
# coll.insert_one(x) # DuplicateKeyError

# insert_one(), replace_one()
* https://docs.mongodb.org/getting-started/python/insert/
* insert_one(), replace_one()의 차이를 구분해 두는 것이 좋다.


# 데이터베이스와 컬렉션 삭제 

In [12]:
# collection내 모든 도큐먼트 삭제
coll.drop()

# collection 삭제
db.drop_collection('mystocks')

# database 삭제
client.drop_database('stocks')


In [13]:
db = client.stocks
mystocks = db.mystocks

# insert_one()

In [14]:
mystocks.insert_one({"Ticker": "AAPL"})
mystocks.insert_one({"Ticker": "GOOG"})
mystocks.insert_one({"Ticker": "XOM"}) 
mystocks.insert_one({"Ticker": "MSFT"})

# find()

In [15]:
mystocks.find_one()

{'Ticker': 'AAPL', '_id': ObjectId('571c13fa9974a86199742481')}

In [16]:
for x in mystocks.find():
    print (x)

{'_id': ObjectId('571c13fa9974a86199742481'), 'Ticker': 'AAPL'}
{'_id': ObjectId('571c13fa9974a86199742482'), 'Ticker': 'GOOG'}
{'_id': ObjectId('571c13fa9974a86199742483'), 'Ticker': 'XOM'}
{'_id': ObjectId('571c13fa9974a86199742484'), 'Ticker': 'MSFT'}


In [17]:
for x in mystocks.find()[:2]:
    print (x)

{'_id': ObjectId('571c13fa9974a86199742481'), 'Ticker': 'AAPL'}
{'_id': ObjectId('571c13fa9974a86199742482'), 'Ticker': 'GOOG'}


In [18]:
for x in mystocks.find()[2:]:
    print (x)

{'_id': ObjectId('571c13fa9974a86199742483'), 'Ticker': 'XOM'}
{'_id': ObjectId('571c13fa9974a86199742484'), 'Ticker': 'MSFT'}


In [19]:
mystocks.count()

4

In [20]:
coll.drop()